In [1]:
import json
import urllib.request
import pandas as pd
import re
import random
import string
from spellchecker import SpellChecker
import numpy as np
from wordle_helpers import (
    words_with_commonly_used_letters,
    words_starting_with_commonly_used_letters,
    sorted_words_by_frequency,
    anagram_pairs,
    anagram_word_frequency,
    anagram_to_keep
)

import os
# where to save the data
ROOT_DIR = "."
FOLDER_NAME = "data"
PATH_TO_FOLDER = os.path.join(ROOT_DIR, FOLDER_NAME)
os.makedirs(PATH_TO_FOLDER, exist_ok=True)

# Load data  

I'll be using the Webster's Unabridged English Dictionary as my source of English words, helpfully curated [here](https://github.com/adambom/dictionary).

In [2]:
with urllib.request.urlopen("https://raw.githubusercontent.com/adambom/dictionary/master/dictionary.json") as url:
    data = json.load(url)

In [ ]:
websters_english_dictionary = pd.DataFrame([data]).T
websters_english_dictionary.reset_index(inplace=True)
websters_english_dictionary.rename(columns={"index": "word", 0: "definition"}, inplace=True)

In [4]:
websters_english_dictionary.head()

,word,definition
0,DIPLOBLASTIC,Characterizing the ovum when it has two primar...
1,DEFIGURE,To delineate. [Obs.]These two stones as they a...
2,LOMBARD,"Of or pertaining to Lombardy, or the inhabitan..."
3,BAHAISM,The religious tenets or practices of the Bahais.
4,FUMERELL,See Femerell.
...,...,...
86031,ELECTRO-MAGNET,"A mass, usually of soft iron, but sometimes of..."
86032,SEAREDNESS,The state of being seared or callous; insensib...
86033,LITHOPHYTOUS,Lithophytic.
86034,FROCKED,Clothed in a frock.


In [ ]:
websters_english_dictionary.info()

In [5]:
websters_english_dictionary.to_csv("./data/websters_english_dictionary.csv", index=False)

In [6]:
all_words = " ".join([word.lower() for word in websters_english_dictionary["word"].to_numpy() if len(word) == 5])

# Create list of five letter words

## Five letter words

In [7]:
# filter out words with special caharacters, eg. hyphens
five_letter_words = re.findall(r"\w+", all_words)

In [8]:
len(five_letter_words)

5387

In [9]:
# Save words as txt file. Text file provided in data folder
with open("./data/webster_dict_all_five_letter_words.txt", mode="w") as file:
    file.write("\n".join(five_letter_words))

## no repeat letter in 5 letter words  

So as to not waste letter guesses, we'll only select words with no repeat letters

In [10]:
no_repeat_letter_five_letter_words = [word for word in five_letter_words if len(set(word)) == 5]
no_repeat_letter_five_letter_words[:10]

['gnarl',
 'arles',
 'stagy',
 'aknow',
 'tawer',
 'copra',
 'knout',
 'atole',
 'timal',
 'dingy']

In [11]:
len(no_repeat_letter_five_letter_words)

3470

## words with most common starting letters  

Further filtering by choosing words form `most_common_starting_letters = "taisocmfpw"`

In [12]:
common_starting_letters = words_starting_with_commonly_used_letters(no_repeat_letter_five_letter_words)
common_starting_letters[:10]

['arles',
 'stagy',
 'aknow',
 'tawer',
 'copra',
 'atole',
 'timal',
 'inkle',
 'moste',
 'wreak']

In [13]:
len(common_starting_letters)

2080

## words frequency

In [14]:
words_sorted_by_frequency = sorted_words_by_frequency(common_starting_letters, sorting=True)
words_sorted_by_frequency[:10]

[('about', 0.004575716203465601),
 ('would', 0.0025520860237645514),
 ('could', 0.002105716897178485),
 ('their', 0.001190727852529245),
 ('maybe', 0.001072666290978991),
 ('after', 0.0010576258274517993),
 ('those', 0.0010403927566496672),
 ('other', 0.0009394233730457645),
 ('first', 0.0009136375616764069),
 ('thing', 0.0008622612539266068)]

In [15]:
len(words_sorted_by_frequency)

2080

# Determine anagrams to remove  

All anagrams in a group will share the same informational content, albeit different positional information. So where we find anagrams we select the one with the highest word frequency. If our guess word is an anagram of the challenge word we'll get yellows and greens.

## Pairs of anagrams

In [16]:
random.seed(44)
paired_anagrams = anagram_pairs([x[0] for x in words_sorted_by_frequency])

In [17]:
paired_anagrams

array([['could', 'cloud'],
       ['those', 'shote'],
       ['those', 'sothe'],
       ...,
       ['muser', 'merus'],
       ['pelta', 'tepal'],
       ['oxfly', 'foxly']], dtype='<U5')

In [18]:
words_sharing_all_letters = pd.DataFrame(paired_anagrams, columns=["word1", "word2"])
words_sharing_all_letters

,word1,word2
0,could,cloud
1,those,shote
2,those,sothe
3,other,throe
4,first,frist
...,...,...
546,suant,astun
547,shote,sothe
548,muser,merus
549,pelta,tepal


In [19]:
words_sharing_all_letters.loc[words_sharing_all_letters["word1"] == "those"]

,word1,word2
1,those,shote
2,those,sothe


In [20]:
words_sharing_all_letters.loc[words_sharing_all_letters["word1"] == "stale"]

,word1,word2
211,stale,satle
212,stale,stela
213,stale,astel


In [21]:
words_sharing_all_letters["word1"].value_counts()

steal    6
stria    5
taper    5
tales    5
plate    5
        ..
widen    1
chute    1
flyer    1
crisp    1
oxfly    1
Name: word1, Length: 387, dtype: int64

In [22]:
anagrams = pd.pivot_table(words_sharing_all_letters,
                          values="word2",
                          columns="word1",
                          index=None,
                          aggfunc=lambda x: ", ".join(x)).T
anagrams.reset_index(inplace=True)
anagrams

,word1,word2
0,abode,adobe
1,abort,tabor
2,acred,cader
3,acrid,caird
4,adept,pated
...,...,...
382,worth,"wroth, whort"
383,wrath,thraw
384,wreak,waker
385,wrote,tower


In [23]:
anagrams["anagrams"] = [", ".join(x) for x in zip(anagrams["word1"], anagrams["word2"])]
anagrams

,word1,word2,anagrams
0,abode,adobe,"abode, adobe"
1,abort,tabor,"abort, tabor"
2,acred,cader,"acred, cader"
3,acrid,caird,"acrid, caird"
4,adept,pated,"adept, pated"
...,...,...,...
382,worth,"wroth, whort","worth, wroth, whort"
383,wrath,thraw,"wrath, thraw"
384,wreak,waker,"wreak, waker"
385,wrote,tower,"wrote, tower"


In [24]:
anagrams.rename(columns={"word1": "word"}, inplace=True)
anagrams.drop(columns=["word2"], inplace=True)

In [25]:
anagrams.loc[(anagrams["word"] == "spare") | (anagrams["word"] == "taper") | (anagrams["word"] == "teams")]

,word,anagrams
277,spare,"spare, spear, parse, prase, asper"
326,taper,"taper, prate, petar, trape, apert, peart"


In [26]:
anagrams.loc[anagrams["word"] == "those"]

,word,anagrams
336,those,"those, shote, sothe"


In [27]:
anagrams.loc[anagrams["word"] == "stale"]

,word,anagrams
287,stale,"stale, satle, stela, astel"


In [28]:
anagrams["word"].nunique()

387

In [29]:
anagrams.to_csv("./data/anagrams.csv", index=False)

## Get word frequency of anagrams

In [30]:
random.seed(123)
anagrams_word_freq = anagram_word_frequency(anagrams["anagrams"].to_numpy())

## Keep anagrams with the highest frequency

In [31]:
anagrams_keep_words = anagram_to_keep(anagrams_word_freq)
anagrams_keep_words[:10]

array(['abode', 'abort', 'acred', 'acrid', 'adept', 'adore', 'afire',
       'aider', 'aitch', 'alert'], dtype=object)

In [32]:
anagrams_keep_words.size

387

## Remove these anagrams

In [33]:
arr_of_words = np.array([x.split(", ") for x in anagrams["anagrams"].to_numpy()], dtype=object)
arr_of_words[:10]

array([list(['abode', 'adobe']), list(['abort', 'tabor']),
       list(['acred', 'cader']), list(['acrid', 'caird']),
       list(['adept', 'pated']), list(['adore', 'oread']),
       list(['afire', 'feria']), list(['aider', 'irade']),
       list(['aitch', 'chati']), list(['alert', 'alter'])], dtype=object)

In [34]:
flat_arr_of_words = np.unique(np.concatenate(arr_of_words))
flat_arr_of_words[:10]

array(['abode', 'abort', 'acerb', 'acred', 'acrid', 'acton', 'adeps',
       'adept', 'adobe', 'adore'], dtype='<U5')

In [35]:
flat_arr_of_words.size

672

In [36]:
anagrams_to_remove = np.setdiff1d(flat_arr_of_words, anagrams_keep_words)

In [37]:
anagrams_to_remove.size

285

In [38]:
anagrams_to_remove[:10]

array(['acerb', 'acton', 'adeps', 'adobe', 'aesir', 'aimer', 'alfet',
       'algin', 'almry', 'alpen'], dtype='<U5')

# Final words

In [39]:
len(common_starting_letters)

2080

In [40]:
common_starting_letters_final_words_idx = \
    (np.isin(np.array(common_starting_letters), anagrams_to_remove, invert=True))

In [41]:
common_starting_letters_final_words_idx

array([ True,  True,  True, ...,  True,  True,  True])

In [42]:
common_starting_letters_final_words = \
    np.array(common_starting_letters)[common_starting_letters_final_words_idx]

In [43]:
common_starting_letters_final_words.size

1795

In [44]:
np.savetxt("./data/webster_common_starting_letters_final_words.txt", common_starting_letters_final_words, fmt="%s")

# OR
# with open("./data/webster_common_starting_letters_final_words.txt", mode="w") as file:
#     file.write("\n".join(common_starting_letters_final_words))